## RFM Analysis 

In [1]:
# General Dependencies
import os
import numpy as np
import pandas as pd

# Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the data

input_csv = input("Enter the input customers file to analyze: ")
file = os.path.join("data", input_csv)

try:
    df_data = pd.read_csv(file)
except:
    print ("File : " + input_csv  + " cannot find in : " + csvpath)
    exit()
    
print('Dataframe dimensions:', df_data.shape)
df_data.describe()

Enter the input customers file to analyze: cust18.csv
Dataframe dimensions: (181, 6)


,NumberOrders,TotalSpent,TotalItems,DaysAsCustomer,DaysNoPurchase
count,181.000000,181.000000,181.000000,181.000000,181.000000
mean,1.762431,480.110497,1.823204,34.867403,27.386740
std,1.503301,694.406788,1.754902,23.496905,23.899434
min,1.000000,41.000000,1.000000,1.000000,1.000000
25%,1.000000,139.000000,1.000000,29.000000,1.000000
50%,1.000000,256.000000,1.000000,29.000000,29.000000
75%,2.000000,457.000000,2.000000,60.000000,60.000000
max,10.000000,5870.000000,14.000000,60.000000,60.000000


In [3]:
df_rfm = df_data.dropna(how='all')
print('Dataframe dimensions:', df_data.shape)

Dataframe dimensions: (181, 6)


In [4]:
df_rfm.head(10)

,CustomerID,NumberOrders,TotalSpent,TotalItems,DaysAsCustomer,DaysNoPurchase
0,DL1004,4,903.0,4.0,29.0,1.0
1,DL1005,2,749.0,2.0,29.0,1.0
2,DL1007,7,1552.0,7.0,60.0,1.0
3,DL1011,1,398.0,1.0,29.0,29.0
4,DL1012,3,3414.0,14.0,29.0,1.0
5,DL1013,4,671.0,4.0,60.0,60.0
6,DL1014,3,1165.0,3.0,29.0,1.0
7,DL1015,3,963.0,3.0,60.0,29.0
8,DL5001,1,1703.0,1.0,1.0,1.0
9,FF1002,2,228.0,2.0,60.0,60.0


## RFM Analysis

RFM (Recency, Frequency, Monetary) analysis is a customer segmentation technique that uses past purchase behavior to divide customers into groups. RFM helps divide customers into various categories or clusters to identify customers who are more likely to respond to promotions and also for future personalization services.

* RECENCY (R): Days since last purchase => df_customers['Days without purchase'] 
* FREQUENCY (F): Total number of purchases => df_customers['Number Orders']
* MONETARY VALUE (M): Total money this customer spent  => df_customers['Total Spent']

In [5]:
quantiles = df_rfm.quantile(q=[0.25,0.5,0.75])
quantiles

,NumberOrders,TotalSpent,TotalItems,DaysAsCustomer,DaysNoPurchase
0.25,1.0,139.0,1.0,29.0,1.0
0.50,1.0,256.0,1.0,29.0,29.0
0.75,2.0,457.0,2.0,60.0,60.0


In [6]:
quantiles.to_dict()

{'DaysAsCustomer': {0.25: 29.0, 0.5: 29.0, 0.75: 60.0},
 'DaysNoPurchase': {0.25: 1.0, 0.5: 29.0, 0.75: 60.0},
 'NumberOrders': {0.25: 1.0, 0.5: 1.0, 0.75: 2.0},
 'TotalItems': {0.25: 1.0, 0.5: 1.0, 0.75: 2.0},
 'TotalSpent': {0.25: 139.0, 0.5: 256.0, 0.75: 457.0}}

In [7]:
# Identify and separate big spenders, lots of orders, long-time customers, 
# dormant customers for sales and marketing campaign use

# RECENCY VALUE (R): Days since last purchase => df_customers['DaysNoPurchase']
# These customers are loyal 

# We calculate Loyal customer base on >= 50% DaysAsCustomer and <= 25% DaysNoPurchase 

def loyal_customer(row):
    if (( row['DaysAsCustomer'] >= quantiles['DaysAsCustomer'][0.50] )  and 
       ( row['DaysNoPurchase'] <= quantiles['DaysNoPurchase'][0.25] )):
        return 4 
    elif (( row['DaysAsCustomer'] >= quantiles['DaysAsCustomer'][0.50] ) and 
         ( row['DaysNoPurchase'] <= quantiles['DaysNoPurchase'][0.50] )):
        return 3
    elif ( ( row['DaysAsCustomer'] >= quantiles['DaysAsCustomer'][0.50] ) and 
        (row['DaysNoPurchase'] <= quantiles['DaysNoPurchase'][0.75])):
        return 2
    elif ((row['DaysAsCustomer'] >= quantiles['DaysAsCustomer'][0.50]) and 
       (row['DaysNoPurchase'] >= quantiles['DaysNoPurchase'][0.75])):
        return 1
    elif ((row['DaysAsCustomer'] >= quantiles['DaysAsCustomer'][0.25]) and 
       (row['DaysNoPurchase'] <= quantiles['DaysNoPurchase'][0.25])):
        return 3
    elif ((row['DaysAsCustomer'] >= quantiles['DaysAsCustomer'][0.25]) and 
       (row['DaysNoPurchase'] <= quantiles['DaysNoPurchase'][0.50])):
        return 2
    else:
        return 1    

In [8]:
# MONETARY VALUE (M): Total money this customer spent => df_customers['Total Spent']

def big_spender(row):
    if row['TotalSpent'] <= quantiles['TotalSpent'][0.25]:
        return 1
    elif row['TotalSpent'] <= quantiles['TotalSpent'][0.50]:
        return 2
    elif row['TotalSpent'] <= quantiles['TotalSpent'][0.75]:
        return 3
    else:
        return 4  

In [9]:
# FREQUENCY VALUE (F):  Total number of purchases => df_customers['Number Orders']

def many_orders(row):
    if row['NumberOrders'] <= quantiles['NumberOrders'][0.25]:
        return 1
    elif row['NumberOrders'] <= quantiles['NumberOrders'][0.50]:
        return 2
    elif row['NumberOrders'] <= quantiles['NumberOrders'][0.75]:
        return 3
    else:
        return 4  

In [10]:
df_rfm['LoyalCustomer'] = df_rfm.apply(loyal_customer, axis=1)
df_rfm['ManyOrders'] = df_rfm.apply(many_orders, axis=1)
df_rfm['BigSpender'] = df_rfm.apply(big_spender, axis=1)

df_rfm['RFMScore'] = df_rfm['LoyalCustomer'].map(str) \
                    + df_rfm['ManyOrders'].map(str) \
                    + df_rfm['BigSpender'].map(str)
df_rfm.head()

,CustomerID,NumberOrders,TotalSpent,TotalItems,DaysAsCustomer,DaysNoPurchase,LoyalCustomer,ManyOrders,BigSpender,RFMScore
0,DL1004,4,903.0,4.0,29.0,1.0,4,4,4,444
1,DL1005,2,749.0,2.0,29.0,1.0,4,3,4,434
2,DL1007,7,1552.0,7.0,60.0,1.0,4,4,4,444
3,DL1011,1,398.0,1.0,29.0,29.0,3,1,3,313
4,DL1012,3,3414.0,14.0,29.0,1.0,4,4,4,444


## How many Customers do we have in each segment?


In [11]:
print("Best Customers: ",len(df_rfm[df_rfm['RFMScore']=='444']))
print('Loyal Customers: ',len(df_rfm[df_rfm['LoyalCustomer']==4]))
print("Big Spenders: ",len(df_rfm[df_rfm['BigSpender']==4]))

almost_lost = len(df_rfm[df_rfm['RFMScore']=='244'])
almost_lost += len(df_rfm[df_rfm['RFMScore']=='233'])
print('Almost Lost: ', almost_lost)

lost_customers = len(df_rfm[df_rfm['RFMScore']=='144'])
lost_customers += len(df_rfm[df_rfm['RFMScore']=='133'])
print('Lost Customers: ',lost_customers)

print('Lost Cheap Customers: ',len(df_rfm[df_rfm['RFMScore']=='111']))

Best Customers:  20
Loyal Customers:  24
Big Spenders:  45
Almost Lost:  3
Lost Customers:  4
Lost Cheap Customers:  6


In [12]:
# Save rfm datasets to csv file for Forecasting

if input_csv == 'cust17.csv' :
    df_rfm.to_csv('data/rfm17.csv',index=False)
elif input_csv == 'cust18.csv' :   
    df_rfm.to_csv('data/rfm18.csv',index=False)
else :
    df_rfm.to_csv('data/rfm.csv',index=False)